# Code

EV aggregator implementation based on State Space Model.

### EV SSM Class

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from ev_ssm import ev_ssm

In [ ]:
sse = ev_ssm(ts=10, N=10000, step=1, tp=100,
             lr=0.1, lp=100, seed=2022, name="EVA")
sse.load_A("Aest.csv")

In [ ]:
sse.g_frc()

In [ ]:
sse.run(tf=10 + 10/3600, disable=False, Pi=0.3)

In [ ]:
sse.plot()

In [ ]:
sse.ev[(sse.ev.u==1) & (sse.ev.c==1)]

### BUG

1. To response a large AGC, all the EVs should be switched to DS, but now some remains IS.
2. The logic of action may need improvement?

In [ ]:
sse = ev_ssm(ts=10, N=10000, step=1, tp=100,
             lr=0.1, lp=100, seed=2022, name="EVA")
sse.load_A("Aests.csv")

u, v, us, vs = sse.r_agc(Pi=5)
u, v, us, vs = sse.r_agc(Pi=23)
print(sse.Pr)
sse.xtab

In [ ]:
sse = ev_ssm(ts=10, N=10000, step=1, tp=100,
             lr=0.1, lp=100, seed=2022, name="EVA")
sse.load_A("Aests.csv")
u, v, us, vs = sse.r_agc(Pi=23)
print(sse.Pr)
sse.xtab

### Test AGC Response

In [ ]:
sse = ev_ssm(ts=10, N=10000, step=1, tp=100,
             lr=0.1, lp=100, seed=2022, name="EVA")
sse.load_A("Aests.csv")

Pi=0
for tf in np.arange(10, 10.1, 1/3600):
    sse.run(tf=tf, Pi=Pi, is_update=True, is_record=True, is_test=False)

In [ ]:
sse.plot()

In [ ]:
sse = ev_ssm(ts=10, N=10000, step=1, tp=100,
             lr=0.1, lp=100, seed=2022, name="EVA")
sse.load_A("Aests.csv")

Pi=0
for tf in np.arange(10, 10.01, 1/3600):
    sse.run(tf=tf, Pi=Pi, is_update=True, is_record=True, is_test=False)

Pi = 0.1
for tf in np.arange(10.01, 10.02, 1/3600):
    sse.run(tf=tf, Pi=Pi, is_update=True, is_record=True, is_test=False)

Pi = 0.05
for tf in np.arange(10.02, 10.03, 1/3600):
    sse.run(tf=tf, Pi=Pi, is_update=True, is_record=True, is_test=False)

Pi = 0.15
for tf in np.arange(10.03, 10.05, 1/3600):
    sse.run(tf=tf, Pi=Pi, is_update=True, is_record=True, is_test=False)

In [ ]:
sse.plot()

## Misc

In [ ]:
u, v, us, vs = sse.r_agc(Pi=5)
print(sse.Pr)
print(us)
print(vs)
sse.xtab

In [ ]:
cdf = pd.DataFrame(np.concatenate([u.reshape(1, 20), v.reshape(1, 20)], axis=0),
                   columns=sse.rtab.columns)

### SAC

This coeef. does not make sense?

In [ ]:
socb = np.arange(0, 1.01, 0.05)
for i in range(len(socb)-1):
    print('soc=', i, 'k=', pow(1-sse.sac(0.5*(socb[i]+socb[i+1]))[2], 20/3600/sse.Th))

In [ ]:
x = np.linspace(0, 1, 100)
sac = sse.sac(x)

plt.style.use('default')
with plt.style.context(['ieee']):
    fig, ax = plt.subplots(1, 1, figsize=(10, 3))
    ax.plot(np.arange(0, 1, 0.01), sac[0], label='CS')
    ax.plot(np.arange(0, 1, 0.01), sac[1], label='IS')
    ax.plot(np.arange(0, 1, 0.01), sac[2], label='DS')
    ax.legend()
    ax.set_ylim([0, 1.01])
    ax.set_xlim([0, 1])
    ax.set_xlabel('SoC [%]')
    ax.set_ylabel('SoC Adapt. Coeff.')

### test

In [ ]:
# sse = ev_ssm(ts=10, N=10000, step=1, tp=100,
#              lr=0.1, lp=100, seed=2022, name="SSE1")

# # --- build and save A matrix ---
# sse.test(tf=10.1)
# sse.save_A("Aests.csv")

# --- load A matrix ---
# sse.load_A("Aests.csv")

# --- run MCS ---
# sse.run(tf=10.1, is_update=True, is_record=True)

# --- plot results ---
# fig, ax = sse.plot()

# --- test A matrix stability ---
# sse.g_res(x0=sse.x0.copy(), n=1000)
# print(np.sum(sse.x0))

# sse.reset(tnow=10, clean_xl=False)

# --- continually run ---
# for tf in np.arange(10, 10.1, 4/3600):
#     sse.run(tf=tf, is_update=True, is_record=True, is_test=False)

# --- spy A ---
# with plt.style.context(['ieee']):
#     fig, ax = plt.subplots(1, 1, figsize=(6, 6), dpi=100)
#     ax.spy(sse.A, markersize=4, marker='o',
#            markerfacecolor='orange', markeredgecolor='orange')
#     ax.grid(which='major', alpha=1)
#     ax.grid(which='minor', alpha=1)
#     ax.set_xlim(0, 3*sse.Ns-1)
#     ax.set_ylim(3*sse.Ns-1, 0)
#     ax.set_title("Spy of mat $A$")
#     ax.minorticks_on()

#     ax.set_xlabel('$\mathbf{x}(k)$')
#     ax.xaxis.set_label_position('top')
#     ax.set_ylabel('$\mathbf{x}(k+1)$')

### EV online numbers

In [3]:
evn = []
sn = []
for t in np.arange(0.0001, 24, 60/3600):
    sse = ev_ssm(ts=t, N=10000, step=1, tp=100,
                 lr=0.1, lp=100, seed=2022, name="EVA")
    evn.append(sse.ev.u.sum())
    sn.append(sse.wsoc)

EVA: ts=0.0001[H], 10000 EVs, Total Q=250.12 MWh
Online 1, Q=0.02 MWh, SoC=0.2692
Power(MW): Pt=-0.006, Pc=-0.006, Pd=-0.0
Ctrl: 1=1; 
EVA: ts=0.0168[H], 10000 EVs, Total Q=250.12 MWh
Online 58, Q=1.45 MWh, SoC=0.3022
Power(MW): Pt=-0.3484, Pc=-0.3484, Pd=-0.0
Ctrl: 1=58; 
EVA: ts=0.0334[H], 10000 EVs, Total Q=250.12 MWh
Online 108, Q=2.71 MWh, SoC=0.3047
Power(MW): Pt=-0.6474, Pc=-0.6474, Pd=-0.0
Ctrl: 1=108; 
EVA: ts=0.0501[H], 10000 EVs, Total Q=250.12 MWh
Online 150, Q=3.76 MWh, SoC=0.309
Power(MW): Pt=-0.8984, Pc=-0.8984, Pd=-0.0
Ctrl: 1=150; 
EVA: ts=0.0668[H], 10000 EVs, Total Q=250.12 MWh
Online 202, Q=5.06 MWh, SoC=0.3102
Power(MW): Pt=-1.2149, Pc=-1.2149, Pd=-0.0
Ctrl: 1=202; 
EVA: ts=0.0834[H], 10000 EVs, Total Q=250.12 MWh
Online 260, Q=6.54 MWh, SoC=0.309
Power(MW): Pt=-1.5634, Pc=-1.5634, Pd=-0.0
Ctrl: 1=260; 
EVA: ts=0.1001[H], 10000 EVs, Total Q=250.12 MWh
Online 319, Q=7.99 MWh, SoC=0.3094
Power(MW): Pt=-1.9188, Pc=-1.9188, Pd=-0.0
Ctrl: 1=319; 
EVA: ts=0.1168[H], 1000

KeyboardInterrupt: 

In [12]:
sse = ev_ssm(ts=18, N=10000, step=1, tp=100,
             lr=0.1, lp=100, seed=2022, name="EVA")

EVA: ts=18[H], 10000 EVs, Total Q=250.12 MWh
Online 2863, Q=71.58 MWh, SoC=0.7623
Power(MW): Pt=-7.9846, Pc=-7.9846, Pd=-0.0
Ctrl: 0=1517; 1=1346; 


In [6]:
sse = ev_ssm(ts=20, N=10000, step=1, tp=100,
             lr=0.1, lp=100, seed=2022, name="EVA")

EVA: ts=20[H], 10000 EVs, Total Q=250.12 MWh
Online 3904, Q=97.57 MWh, SoC=0.8417
Power(MW): Pt=-7.4659, Pc=-7.4659, Pd=-0.0
Ctrl: 0=2647; 1=1257; 


In [ ]:
with plt.style.context(['science', 'ieee']):
    fig_evn, ax_evn = plt.subplots(figsize=(1.5, 1.0))
    ax_evn.plot(sse.tss, evn)
    ax_evn.grid()
    ax_evn.set_xlim([0, 24])
    ax_evn.set_ylim(bottom=0)

In [ ]:
with plt.style.context(['science', 'ieee']):
    fig_evn, ax_evn = plt.subplots(figsize=(1.5, 1.0))
    ax_evn.plot(sse.tss, sn)
    ax_evn.grid()
    ax_evn.set_xlim([0, 24])
    ax_evn.set_ylim(bottom=0)

### Spy An

In [ ]:
# Long enough, all the EVs will become idle.

An = np.linalg.matrix_power(sse.A, 225)

# --- spy A ---
with plt.style.context(['ieee']):
    fig, ax = plt.subplots(1, 1, figsize=(6, 6), dpi=100)
    ax.spy(An, markersize=4, marker='o',
           markerfacecolor='orange', markeredgecolor='orange')
    ax.grid(which='major', alpha=1)
    ax.grid(which='minor', alpha=1)
    ax.set_xlim(0, 3*sse.Ns-1)
    ax.set_ylim(3*sse.Ns-1, 0)
    ax.set_title("Spy of mat $A$")
    ax.minorticks_on()

    ax.set_xlabel('$\mathbf{x}(k)$')
    ax.xaxis.set_label_position('top')
    ax.set_ylabel('$\mathbf{x}(k+1)$')

In [ ]:
# control vector validation

cl = -1*sse.rtab.iloc[1].values - sse.rtab.iloc[2].values
cu = sse.rtab.iloc[0].values + sse.rtab.iloc[1].values

u = np.zeros((20,))
v = np.zeros((20,))

# max output
u = sse.rtab.iloc[0].values
v = sse.rtab.iloc[1].values + sse.rtab.iloc[0].values

# min output
u = -1 * sse.rtab.iloc[1].values - sse.rtab.iloc[2].values
v = -1 * sse.rtab.iloc[2].values

res = sse.x0 + np.matmul(sse.B, u).reshape(-1) + np.matmul(sse.C, v).reshape(-1)

print("res check:", np.all(res >= 0))

with plt.style.context(['ieee']):
    width = 0.35
    x = np.arange(len(sse.x0))
    fig, ax = plt.subplots(1, 1, figsize=(12, 5))
    ax.bar(x - 0.5*width, sse.x0, width, label="in", color='cyan')
    ax.bar(x + 0.5*width, res, width, label="out", color='orange')
    ax.set_xlim(0, 59)
    ax.set_ylim(-0.0, 0.01)
    ax.legend(['x0', 'x0+ctrl'])